## First step to clean the dataset:

- Select only English movies
- Select onlu full-length feature (movies of more than one hour long)
- Remove movies for which we don't have the title

In [5]:
import pandas as pd
#import requests
import numpy as np
import string
import json

In [6]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [8]:
#Copy of movie_metadata dataset
movie_metadata_filtered = movie_metadata.copy(deep= True)

# Make the colomn Movie_languages, Movie_countries and Movie_genres more readable

def readable(colomn):

    pd.set_option('mode.chained_assignment', None) #remove warning

    for i in colomn.index:
        d=json.loads(colomn[i]) #change set of each row from the colomn to dict
        colomn[i]=list(d.values()) #change the dict into list

readable(movie_metadata_filtered['Movie_languages'])
readable(movie_metadata_filtered['Movie_countries'])
readable(movie_metadata_filtered['Movie_genres'])

movie_metadata_filtered.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama]
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"[Silent film, English Language]",[United States of America],"[Short Film, Silent film, Indie, Black-and-whi..."
6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,[English Language],[South Africa],"[Family Film, Fantasy, Adventure, World cinema]"
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,[English Language],[United States of America],"[Musical, Comedy, Black-and-white]"
8,9997961,/m/06_y2j7,Contigo y aquí,1974,NaN,NaN,[Spanish Language],[Argentina],"[Musical, Drama, Comedy]"
9,2345652,/m/075f66,City of the Dead,1960,NaN,76.0,[English Language],[United Kingdom],"[Horror, Supernatural]"


In [9]:

#Filtering of movie_metada, keeping only full-length features movies (more than 60 minutes long), in English and with that have a Title in the database.

movie_metadata_filtered =movie_metadata_filtered[movie_metadata_filtered.Movie_languages.map(set(['English Language']).issubset)]

movie_metadata_filtered.dropna(subset=['Movie_name'],inplace=True)

movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_runtime >= 60]

movie_metadata_filtered.dropna(subset=['Movie_release_date'], inplace = True)
movie_metadata_filtered["Movie_release_date"] = movie_metadata_filtered['Movie_release_date'].apply(lambda x : str(x)[:4]).apply(lambda x : int(x))
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_release_date <= 2012]

print(movie_metadata_filtered)

movie_English_first = movie_metadata_filtered[movie_metadata_filtered.Movie_languages.str[0]=="English Language"] #English Language as First Language

movie_English_only = movie_metadata_filtered[movie_metadata_filtered.Movie_languages.map(set(['English Language']).issuperset)] #Only English Language


       Wikipedia_movie_ID Freebase_movie_ID  \
0                  975900         /m/03vyhn   
1                 3196793         /m/08yl5d   
3                 9363483        /m/0285_cd   
6                18998739        /m/04jcqvw   
7                10408933        /m/02qc0j7   
...                   ...               ...   
81735            32468537        /m/0crwd9y   
81736            35228177        /m/0j7hxnt   
81737            34980460        /m/0g4pl34   
81738             9971909        /m/02pygw1   
81740            12476867        /m/02w7zz8   

                                              Movie_name  Movie_release_date  \
0                                         Ghosts of Mars                2001   
1      Getting Away with Murder: The JonBenét Ramsey ...                2000   
3                                       White Of The Eye                1987   
6                              The Sorcerer's Apprentice                2002   
7                               Ale

In [10]:
movie_English_only.to_csv(path_or_buf = "Data/movies_metadata_english_only.csv", sep=',')

In [8]:
# IMDB files 

ratings_imdb = pd.read_table('Data/ratings_imdb.tsv', sep="\t")
ratings_imdb.head()


movies_imdb= pd.read_table('Data/movies_imdb.tsv', sep="\t")



In [9]:
# Rotten tomatoes 

rotten_tomatoes = pd.read_table('Data/movies_rottentomatoes.csv', sep=",")
rotten_tomatoes.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [11]:
# Genre processing: give only one genre to our films, preferably the main genre (first on the list), from a "basic" dictionnary of genre


# 1st step: create a new genre dictionnary
dict_genre = {
    "Action": ["Action/adventure", "ninja movie", "epic", "auto racing", "martial arts film","Combat Films", "race movie", "samurai cinema", "Super hero", "superhero movie", "superhero"],
    "Adventure": ["Adventure", "travel"],
    "Art": ["Concert film", "Film-Opera", "Operetta", "Punk rock", "Rockumentary", "Space opera", "Singing cowboy", "Hip hop movies", "Breakdance", "Dance", "musical", "& Literature", "Literature", "Music"],
    "Comedy": ["Humour", "Tragicomedy", "Ealing Comedies", "Comdedy", "Adventure Comedy", "Commedy of manners", "Workplace comedy", "comedy-drama", "Buddy Picture", "Buddy cop", "Parody"], 
    "Documentary": ["Political Documetary", "Docudrama", "Historical Documentaries", "Rockumentary", "media studies", "media satire", "Political cinema", "Political Documentary"],
    "Drama": ["Melodrama", "Tragedy", "Tragicomedy", "Docudrama", "Family drama", "Comedy-drama", 'illnesses & disabilities'], 
    "Family": ["Family Film", "Animation", "Animated cartoon", "Children's film", "children's", "children's entertainment", "children's fantasy", "children's/family", "For children"], 
    "Fiction": ["fairy tail", "fictional film", "medical fiction", "mythological fantasy", "sci fi pictures", 'sci fi pictures original films', "sword and sorcery films", "sword and sandal", "time travel", "supernatural", "Alien invasion","Alien Film", "Zombie film", "Z-movie", "z movie", "Zombie"],
    "Historical": ["history", "Biographical", "British History", "British Empire Film", "British new wave", "Historical Epic", "Cold War", "Historical Documentaries", "Biography", "Biopic [feature]", "The Netherlands in World War II", "Gulf War", "movies about gladiators"],
    "Horror": ["Natural Horror film", "Sci-fi Horror", "road-horror", "Creature Movie", "Monster Movie", "Creature"],
    "Porn": ["Pornographic movie", "Pornography", "Softcore Porn", "Adult", "Sexploitation", "Erotica", "homoeroticism", "erotic thriller", "Hardcore pornography", "Homoeroticism"], 
    "Romance": ["Romance film", "romantic fantasy"],
    "Sport": ["baseball", "boxing", "sports"],
    "Thriller": [ "film à clef", "mystery", "suspense", 'crime', 'crime fiction', 'crime thriller', "master criminal films", "detective", "Master Criminal Films", "Heist", "Gangster Film"],
    "Western": ["Singing cowboy", "Westerns", "B-Western", "Epic Western", "Spaghetti Western"],
    "World cinema": ["Chinese Movies", "Japanese Movies", "Latino", "americana", "bengali cinema", "filipino movies", "Anime", "Indie"],
    "War film": ["war", "War effort", "anti-war", "anti-war film", "nuclear warfare", "foreign legion"],

}

In [36]:
# 2nd step: Revert the current dictionnary, easier to manipulate  À GARDER?

dict_genre_revert = {}
for key, value in dict_genre.items():
    key = key.lower() #passage en minuscule
    for v in value:
        v = v.lower()
        dict_genre_revert[v] = key
    dict_genre_revert[key] = key

dict_genre_revert

{'action/adventure': 'action',
 'ninja movie': 'action',
 'epic': 'action',
 'auto racing': 'action',
 'martial arts film': 'action',
 'combat films': 'action',
 'race movie': 'action',
 'samurai cinema': 'action',
 'super hero': 'action',
 'superhero movie': 'action',
 'superhero': 'action',
 'action': 'action',
 'adventure': 'adventure',
 'travel': 'adventure',
 'concert film': 'art',
 'film-opera': 'art',
 'operetta': 'art',
 'punk rock': 'art',
 'rockumentary': 'documentary',
 'space opera': 'art',
 'singing cowboy': 'western',
 'hip hop movies': 'art',
 'breakdance': 'art',
 'dance': 'art',
 'musical': 'art',
 '& literature': 'art',
 'literature': 'art',
 'music': 'art',
 'art': 'art',
 'humour': 'comedy',
 'tragicomedy': 'drama',
 'ealing comedies': 'comedy',
 'comdedy': 'comedy',
 'adventure comedy': 'comedy',
 'commedy of manners': 'comedy',
 'workplace comedy': 'comedy',
 'comedy-drama': 'drama',
 'buddy picture': 'comedy',
 'buddy cop': 'comedy',
 'parody': 'comedy',
 'comedy

In [ ]:
#inspiration:  
def get_genre(set_original_genres):
    set_new_genres = set()
    for g in set_original_genres:
        g = g.lower()
        temp = True
        for s_g in g.split(" "): 
            if s_g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[s_g])
                temp = False
        if temp:
            if g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[g])
    return set_new_genres

In [96]:
# 3rd step:-regarder le 1er genre de la liste original, si dans clef du dict reverse, attribuer le nouveau genre correspondant
#          -si 1er genre pas dans clef, passer au 2ème genre de la liste (et faire la même chose)
#          -si aucun genre ne correspond, print la ligne

movie_EF_main_genre = movie_English_first.copy(deep=True)

movie_EF_main_genre['Movie_main_genre']=0



In [105]:
for i in movie_EF_main_genre.index:
    N=len(movie_EF_main_genre['Movie_genres'][i])
    k=0
    while k < N:
        genre = movie_EF_main_genre['Movie_genres'][i][k].lower()
        if genre in dict_genre_revert.keys():
            movie_EF_main_genre['Movie_main_genre'][i]= dict_genre_revert.get(genre)
            print(dict_genre_revert.get(genre))
            k=N+1
        else:
            k+=1
    if k==N:
       movie_EF_main_genre['Movie_main_genre'][i]= "ALED"

movie_EF_main_genre[movie_EF_main_genre['Movie_main_genre']=="ALED"]

thriller
thriller
thriller
family
art
horror
drama
drama
war film
family
drama
thriller
drama
thriller
romance
comedy
art
comedy
thriller
war film
drama
romance
historical
thriller
historical
family
action
horror
romance
action
porn
comedy
romance
thriller
thriller
drama
documentary
thriller
horror
comedy
horror
thriller
historical
documentary
drama
historical
drama
thriller
action
drama
romance
horror
drama
thriller
thriller
art
thriller
drama
world cinema
thriller
art
horror
action
horror
world cinema
thriller
art
thriller
thriller
drama
family
drama
drama
thriller
romance
art
historical
historical
adventure
drama
horror
romance
romance
thriller
action
thriller
documentary
adventure
thriller
documentary
action
comedy
adventure
drama
horror
drama
drama
horror
historical
thriller
thriller
romance
porn
sport
art
romance
thriller
thriller
thriller
historical
thriller
fiction
horror
drama
action
drama
thriller
action
thriller
drama
western
drama
comedy
drama
thriller
romance
romance
drama

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Movie_main_genre
113,28301986,/m/0cnxjzx,Dig That Uranium,1956,NaN,61.0,[English Language],[United States of America],[Comedy film],ALED
116,3740032,/m/09yd9c,Desperate,1947,NaN,73.0,[English Language],[United States of America],[Film noir],ALED
653,12632216,/m/02wywht,The Two Worlds of Jenny Logan,1979,NaN,94.0,[English Language],[United States of America],[Romantic drama],ALED
657,33096336,/m/0h66h0p,"Jeff, Who Lives at Home",2011,4269426.0,83.0,[English Language],[United States of America],[Comedy film],ALED
692,29119625,/m/0dlmyyq,Canaries Sometimes Sing,1930,NaN,80.0,[English Language],[United Kingdom],[Comedy film],ALED
...,...,...,...,...,...,...,...,...,...,...
81083,6214470,/m/0fx586,Arcanum,2006,NaN,80.0,[English Language],[Canada],[Science Fiction],ALED
81348,20887118,/m/05b32x8,Battlestar Galactica: The Plan,2009,NaN,112.0,[English Language],[United States of America],"[Science Fiction, Sci-Fi Adventure]",ALED
81563,25655938,/m/09v7p4g,I Was a Mail Order Bride,1982,NaN,92.0,[English Language],[United States of America],[Romantic comedy],ALED
81583,35028178,/m/0j62vp8,Fast Charlie... the Moonbeam Rider,1979,444334.0,98.0,[English Language],[United States of America],[Comedy film],ALED


# preprocess character metadata before to merge it with movies



In [17]:
# Ajouter les noms des personnages principaux au movie.metadata

character_metadata_copy= character_metadata.copy(deep= True)
character_metadata_copy.dropna(subset=['Character_name'], inplace=True)
#il reste encore des NaN !!!!

movie_character = movie_English_only.merge(character_metadata_copy, how='left', left_on='Wikipedia_movie_ID', right_on='Wikipedia_movie_ID')

movie_character = movie_character[['Wikipedia_movie_ID', 'Movie_name',  'Movie_box_office_revenue', 'Movie_runtime', 'Movie_languages', 'Movie_countries', 'Movie_genres', 'Character_name', 'Freebase_character_ID']]
# je sais pas pourquoi mais il veut pas mettre le wikipedia movie ID et la release date

grouped_movie_character = movie_character.groupby('Wikipedia_movie_ID').agg({
    'Movie_name': 'first',  
    'Movie_box_office_revenue': 'first', 
    'Movie_runtime': 'first', 
    'Movie_languages': 'first', 
    'Movie_countries': 'first',  
    'Movie_genres': 'first',  
    'Character_name': lambda x: ', '.join(x.dropna()),  
    'Freebase_character_ID':  lambda x: ', '.join(x.dropna())
}).reset_index()
grouped_movie_character.sample(5).head()

,Wikipedia_movie_ID,Movie_name,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres,Character_name,Freebase_character_ID
17002,17199185,The Man with Nine Lives,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",,
3926,1996645,Old Gringo,3574256.0,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/03k9fj"": ""Adventure"", ""/m/04xvlr"": ""Perio...","Harriet Winslow, Gen. Tomas Arroyo","/m/0gycxzc, /m/0h3b1sj"
13949,11774920,Kissing Cousins,NaN,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/02l7c8"": ""...",Amir,/m/0h71kf1
23914,29259863,Timbuctoo,NaN,72.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/05p553"": ""Comedy film""}",,
24921,31756842,Elena Undone,NaN,105.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hn10"": ""LGBT"", ""/m/02l7c8"": ""Romance Fil...","Elena, Peyton, Barry, Tyler, Nash, Tori, Wave,...","/m/0bzb7sx, /m/0bzb7t4, /m/0bzb7t8, /m/0bzb7tk..."


In [18]:
# check if one of the character's name is present in the movie title

movie_character['Character_in_title'] = movie_character.apply(lambda row: str(row['Character_name']).lower() in str(row['Movie_name']).lower(), axis=1)
movie_with_character_in_title = movie_character['Character_in_title'].sum()


print(f"Fraction of movie titles containing character name: {movie_with_character_in_title / len(movie_character):.2%}")
print(movie_character.loc[movie_character['Character_in_title'] == 1, ['Movie_name', 'Character_name']])

Fraction of movie titles containing character name: 1.21%
                                     Movie_name   Character_name
29                                      Henry V          Henry V
38                                 Mary Poppins     Mary Poppins
200                 ChromeSkull: Laid to Rest 2      ChromeSkull
272                      Good Morning Miss Dove        Miss Dove
399                                   RoboCop 3          RoboCop
...                                         ...              ...
108887                             Stingray Sam     Stingray Sam
109242                                    Emile            Emile
109308                            Grandma's Boy    Grandma's Boy
109768  Sherlock Holmes and the Voice of Terror  Voice of Terror
109777  Sherlock Holmes and the Voice of Terror  Sherlock Holmes

[1326 rows x 2 columns]


In [19]:
# certains noms de personnages ne sont aps vraiment des prénoms, est-ce qu'il faudra faire de la NLP?
